In [22]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse
files = pd.read_csv("all_games_updated.csv")

In [61]:
def time_weighted_average(rows, statistic):
    stat = rows[statistic]
    if (rows.shape[0] == 5):
        return .297*stat.iloc[4,] + .297*.8*stat.iloc[3,] + .297*(.8**2)*stat.iloc[2,] + .297*(.8**3)*stat.iloc[1,] + .297*(.8**4)*stat.iloc[0,]
    elif (rows.shape[0] == 10):
        return .224*stat.iloc[9,] + .224*.8*stat.iloc[8,] + .224*(.8^2)*stat[7] + .224*(.8^3)*stat[6] + .224*(.8^4)*stat[5] + .224*(.8^5)*stat[4] + .224*(.8^6)*stat[3] + .224*(.8^7)*stat[2] + .297*(.8^4)*stat[0]

In [37]:
def get_stats(player, date, number_rows):
    converted_datetime = datetime.strptime(date, '%Y-%m-%d')
    player_rows = files.loc[files['name'] == player]
    selected_rows = []
    if (len(player_rows)) < number_rows:
        for i in range(len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
        return pd.DataFrame(selected_rows)
    index = 0
    for i in range(len(player_rows)):
        curr_date = player_rows.iloc[i]['Date']
        if datetime.strptime(curr_date, '%Y-%m-%d') >= converted_datetime:
            index = i - 1
            break
    if index != 0:
        if index + 1 - number_rows < 0:
            for i in range(index + 1):
                selected_rows.append(player_rows.iloc[i])
        else:
            for i in range(index + 1 - number_rows, index + 1):
                selected_rows.append(player_rows.iloc[i])
#     elif converted_datetime >= players_rows[len(player_rows) - 1]:
    else:
        for i in range(len(player_rows) - number_rows, len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
    return pd.DataFrame(selected_rows)

In [16]:
player_box_scores = pd.read_csv('all_games_updated.csv')

input_statistics = ["made_field_goals", "made_two_point_field_goals", "attempted_two_point_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals"]
for box_index in player_box_scores.index:
    box_score = player_box_scores.loc[box_index]
    player_name = box_score["name"]
    game_date = box_score["Date"]
    last_5_rows = get_stats(player_name, game_date, 5)
    weighted_stats = []
    for stat in input_statistics:
        weighted_stats.append(time_weighted_average(last_5_rows, stat))
    line_with_averages = 

SyntaxError: invalid syntax (<ipython-input-16-dd5ce807971f>, line 4)

In [62]:
for box_index in files.index[10021:10040]:
    boxscore = files.loc[box_index]
    print(time_weighted_average(get_stats(boxscore["name"], boxscore["Date"], 5), "assists"))

2.9524176000000004
1.2098592000000001
0.0
1.8922463999999999
2.4501312
0.4186512
0.6329664
0.449064
3.7084608
1.4450832000000002
1.21176
3.3819984000000005
4.9648896
1.10484
1.4170463999999998
0.7318080000000001
2.4216192000000003
4.363286400000001
1.7834256000000002


RangeIndex(start=1, stop=10, step=1)

In [54]:
files.shape[0]

12757